In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


# #Data Science project:Data Wrangling
This will demonstrate my walkthrough data wrangling project using python
The steps we will go through includes:
 * Data Discovery
 * Data Structuring
* Data Cleaning
* Data Validation
* Data Publishing
   

1.Data Discovery

In [2]:
#Import the Data to a Pandas DataFrame
df=pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
#Have a quick overview of the data
df.info()
# Number of rows and columns
print("Shape of the dataset (R x C):", df.shape)
# List of all column names
print("Columns in the dataset:\n", df.columns.tolist())
# Data types of each column
print("Data types:\n", df.dtypes)
# Group and Count of missing (null) values in each column
print("Missing values per column:\n", df.isnull().sum())
#Group and Count the  duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())
# Group and Count of duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
Shape of the dataset (R x C): (8807, 12)
Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Data types:
 show_id         object
type      

2.	Data Structuring

In [3]:
# Convert 'date_added' to datetime
df['date_added'] = pd.to_datetime(df['date_added'],format='mixed')
# Separate 'duration' into numeric value and unit (e.g., '90 min' → 90, 'min')
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')

# Convert duration_value to numeric
df['duration_value'] = pd.to_numeric(df['duration_value'])

# View Resulting columns
print(df[['duration_value', 'duration_unit']])

      duration_value duration_unit
0               90.0           min
1                2.0       Seasons
2                1.0        Season
3                1.0        Season
4                2.0       Seasons
...              ...           ...
8802           158.0           min
8803             2.0       Seasons
8804            88.0           min
8805            88.0           min
8806           111.0           min

[8807 rows x 2 columns]


Cleaning

In [4]:
# Check for duplicate rows
print("Duplicate rows before:", df.duplicated().sum())

# Drop duplicate rows if any
df = df.drop_duplicates()

# Drop description column because it will not be used
df = df.drop(columns=['description'])

# Impute Director values by using relationship between cast and director

# List of Director-Cast pairs and the number of times they appear
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts() #counts unique values
filtered_counts = counts[counts >= 3] #checks if repeated 3 or more times
filtered_values = filtered_counts.index #gets the values i.e. names
lst_dir_cast = list(filtered_values) #convert to list
dict_direcast = dict()
for i in lst_dir_cast :
        director,cast = i.split('---’)
            dict_direcast[director]=cast
for i in range(len(dict_direcast)): 
    df.loc[(df['director'].isna()) & (df['cast'] == list(dict_direcast.items())[i][1]),'director'] = list(dict_direcast.items())[i][0]

# Assign Not Given to all other director fields
df.loc[df['director'].isna(),'director'] = 'Not Given’

#Use directors to fill missing countries
directors = df['director’]
countries = df['country’]
#pair each director with their country use zip() to get an iterator of tuples
pairs = zip(directors, countries)
# Convert the list of tuples into a dictionary
dir_cntry = dict(list(pairs))

# Director matched to Country values used to fill in null country values
for i in range(len(dir_cntry)):    
df.loc[(df['country'].isna()) & (df['director'] == list(dir_cntry.items())[i][0]),'country'] = list(dir_cntry.items())[i][1]
# Assign Not Given to all other country fields
df.loc[df['country'].isna(),'country'] = 'Not Given'

# Assign Not Given to all other fields
df.loc[df[‘cast'].isna(),’cast'] = 'Not Given'

# dropping other row records that are null
df.drop(df[df['date_added'].isna()].index,axis=0,inplace=True)
df.drop(df[df['rating'].isna()].index,axis=0,inplace=True)
df.drop(df[df['duration'].isna()].index,axis=0,inplace=True)

Errors
# check if there are any added_dates that come before release_year
import datetime as dt
sum(df['date_added'].dt.year < df['release_year’])
df.loc[(df['date_added'].dt.year < df['release_year']),['date_added','release_year’]]
# sample some of the records and check that they have been accurately replaced
df.iloc[[1551,1696,2920,3168]]
#Confirm that no more release_year inconsistencies
sum(df['date_added'].dt.year < df['release_year'])

SyntaxError: unterminated string literal (detected at line 20) (364497544.py, line 20)

Validate


In [ ]:
#Remove any columns you may have added during wrangling e.g.
df.drop(columns=['dir_cast'], inplace=True)
#Check the consistency, accuracy, and completeness of the data
#Ensure each column has the correct data type e.g. verify that date_added is datetime and duration_value is numeric.
#Use business logic or sanity rules to identify anomalies e.g. records before 1997
#Ensure no important fields are still missing
#Sample a few rows to check visually e.g. 
df.sample(5)
#Reset the Index e.g. 
df_reset = df.reset_index(drop=True)


Publish

In [ ]:
# Save as CSV 
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False)